In [1]:
import sys
sys.path.append("..")

import logging
import random
import neptune
import numpy as np
import syft as sy
from torch import load
import torch

from torchvision import transforms
from federated_learning.FLCustomDataset import FLCustomDataset
from federated_learning.FederatedLearning import FederatedLearning
from federated_learning.helper import utils

CONFIG_PATH = '../configs/defaults.yml'
configs = utils.load_config(CONFIG_PATH)
arguments = dict()
arguments['--reg'] = 0.0
arguments['--output-prefix'] = "mnist_no_attack"
arguments['--server_model'] = "data_output/20201107_030649_mnist_w0/server_model_9"
arguments["--epoch"] = 1
arguments["--round"] = 1
output_dir = None
log_enable = False
neptune_enable = False


configs = utils.load_config(CONFIG_PATH)
logging.basicConfig(format='%(asctime)s %(message)s', level=configs['log']['level'])
random.seed(configs['runtime']['random_seed'])

# From command line
epochs_num = int(arguments["--epoch"])
rounds_num = int(arguments["--round"])


fl = FederatedLearning(
    configs['runtime']['batch_size'], 
    configs['runtime']['test_batch_size'], 
    configs['runtime']['lr'], 
    float(arguments['--reg']) if arguments['--reg'] is not None else 0.0,
    configs['runtime']['momentum'], 
    neptune_enable, log_enable, 
    configs['log']['interval'], 
    output_dir, 
    configs['runtime']['random_seed'])


fl.create_server()
fl.create_server_model()

total_num_workers = \
        configs['runtime']['mnist_normal_users_num'] + \
        configs['runtime']['mnist_eavesdropper_num'] 
        # configs['runtime']['mnist_trusted_users_num']

workers_idx = ["worker_" + str(i) for i in range(total_num_workers)]
fl.create_workers(workers_idx)
fl.create_workers_model(workers_idx)

# trusted_idx = utils.get_workers_idx(
#     range(total_num_workers), configs['runtime']['mnist_trusted_users_num'], [])
eavesdroppers_idx = utils.get_workers_idx(
    range(total_num_workers), configs['runtime']['mnist_eavesdropper_num'], [])
normal_idx = utils.get_workers_idx(
    range(total_num_workers), configs['runtime']['mnist_normal_users_num'], eavesdroppers_idx)

# trusted_idx = [workers_idx[ii] for ii in trusted_idx]
eavesdroppers_idx = [workers_idx[ii] for ii in eavesdroppers_idx]
normal_idx = [workers_idx[ii] for ii in normal_idx]

# logging.info("Trusted: {}".format(trusted_idx))
logging.info("Eavesdroppers: {}".format(eavesdroppers_idx))
logging.info("Normal: {}".format(normal_idx))
if log_enable:
    # utils.write_to_file(output_dir, "trusted", trusted_idx)
    utils.write_to_file(output_dir, "eavesdroppers", eavesdroppers_idx)
    utils.write_to_file(output_dir, "normal", normal_idx)

train_raw_dataset = utils.preprocess_mnist(
    utils.load_mnist_data_train(
        configs['data']['MNIST_PATH'], 
        configs['runtime']['mnist_data_percentage']))
train_dataset = utils.get_mnist_dataset(train_raw_dataset)

test_data = utils.load_mnist_data_test(configs['data']['MNIST_PATH'])
test_dataset = utils.get_mnist_dataset(test_data)
test_dataloader = utils.get_dataloader(
    test_dataset, configs['runtime']['test_batch_size'], shuffle=True, drop_last=False)

# W0 model
trained_w0_model = load(configs['runtime']['W0_pure_path'])

federated_train_dataloader = None


Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/ubuntu/.local/lib/python3.6/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


2020-12-02 19:14:14,156 Initializing Federated Learning class...
2020-12-02 19:14:14,920 Creating the server...
2020-12-02 19:14:14,922 Creating a model for the server...
2020-12-02 19:14:14,931 Creating workers...
2020-12-02 19:14:14,934 Creating a model for 30 worker(s)...
2020-12-02 19:14:15,094 Eavesdroppers: ['worker_13', 'worker_23', 'worker_0', 'worker_26', 'worker_25']
2020-12-02 19:14:15,095 Normal: ['worker_9', 'worker_27', 'worker_11', 'worker_29', 'worker_6', 'worker_8', 'worker_18', 'worker_5', 'worker_3', 'worker_17', 'worker_20', 'worker_19', 'worker_2', 'worker_16', 'worker_4', 'worker_10', 'worker_14', 'worker_22', 'worker_21', 'worker_24', 'worker_28', 'worker_12', 'worker_7', 'worker_1', 'worker_15']
2020-12-02 19:14:15,097 Loading 30% of train data from MNIST dataset.
2020-12-02 19:14:15,234 Preparing the MNIST dataset.
2020-12-02 19:14:15,266 Creating MNIST dataset.
2020-12-02 19:14:15,321 Loading test data from MNIST dataset.
2020-12-02 19:14:15,343 Creating MNIST

In [2]:
ww = fl.find_best_weights(trained_w0_model, workers_idx)

2020-12-02 19:14:16,335 (500, 30)
2020-12-02 19:14:16,336 (20, 30)
2020-12-02 19:14:16,355 (25000, 30)
2020-12-02 19:14:16,357 (50, 30)
2020-12-02 19:14:17,190 (400000, 30)
2020-12-02 19:14:17,193 (500, 30)
2020-12-02 19:14:17,199 (5000, 30)
2020-12-02 19:14:17,200 (10, 30)
2020-12-02 19:14:17,798 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
2020-12-02 19:14:17,801 


In [3]:
ww

array([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,  1.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0.])